In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os 
import time
import time

In [2]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

INFO:tensorflow:Initializing the TPU system: grpc://10.90.53.234:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.90.53.234:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU')]


In [3]:
strategy = tf.distribute.TPUStrategy(resolver)

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [4]:
BATCH_SIZE = 128 * strategy.num_replicas_in_sync # Gobal batch size.
# The global batch size will be automatically sharded across all the 8 cores
LEARNING_RATE = 0.01

training_images_file   = 'gs://mnist-public/train-images-idx3-ubyte'
training_labels_file   = 'gs://mnist-public/train-labels-idx1-ubyte'
validation_images_file = 'gs://mnist-public/t10k-images-idx3-ubyte'
validation_labels_file = 'gs://mnist-public/t10k-labels-idx1-ubyte'

In [5]:
def read_label(tf_bytestring):
    label = tf.io.decode_raw(tf_bytestring, tf.uint8)
    label = tf.reshape(label, [])
    label = tf.one_hot(label, 10)
    return label

In [6]:
def read_image(tf_bytestring):
    image = tf.io.decode_raw(tf_bytestring, tf.uint8)
    image = tf.cast(image, tf.float32)/255.0
    image = tf.reshape(image, [28*28])
    return image

In [7]:
def load_dataset(image_file, label_file):
    imagedataset = tf.data.FixedLengthRecordDataset(image_file, 28*28, header_bytes=16, num_parallel_reads=tf.data.experimental.AUTOTUNE) #loads the length and the reads the file simultaneously
    imagedataset = imagedataset.map(read_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)  #reads the function which was used for preprocessin
    labelsdataset = tf.data.FixedLengthRecordDataset(label_file, 1, header_bytes=8, num_parallel_reads=tf.data.experimental.AUTOTUNE)
    labelsdataset = labelsdataset.map(read_label, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = tf.data.Dataset.zip((imagedataset, labelsdataset))
    return dataset 

In [8]:
def get_training_dataset(image_file, label_file, batch_size):
    dataset = load_dataset(image_file, label_file)
    dataset = dataset.cache()  # dataset can be entirely cached in RAM
    dataset = dataset.shuffle(5000, reshuffle_each_iteration=True) #shuffling the dataset
    dataset = dataset.repeat() 
    dataset = dataset.batch(batch_size, drop_remainder=True) # drop_remainder makes sure we have the exact batch
    dataset = dataset.prefetch(-1)  # fetch next batches while training on the current one 
    return dataset

In [9]:
def get_validation_dataset(image_file, label_file):
    dataset = load_dataset(image_file, label_file)
    dataset = dataset.cache() #  dataset can be entirely cached in RAM
    dataset = dataset.batch(10000, drop_remainder=True) # 10000 items in eval dataset, all in one batch
    dataset = dataset.repeat() 
    return dataset

In [10]:
training_dataset = get_training_dataset(training_images_file, training_labels_file, BATCH_SIZE)
validation_dataset = get_validation_dataset(validation_images_file, validation_labels_file)

In [11]:
def create_model():
  model = tf.keras.Sequential(
      [
        tf.keras.layers.Reshape(input_shape=(28*28,), target_shape=(28, 28, 1), name="image"),

        tf.keras.layers.Conv2D(filters=12, kernel_size=3, padding='same'), # no bias necessary before batch norm
        tf.keras.layers.Activation('relu'), # activation after batch norm

        tf.keras.layers.Conv2D(filters=24, kernel_size=6, padding='same'),
        tf.keras.layers.Activation('relu'),

        tf.keras.layers.Conv2D(filters=32, kernel_size=6, padding='same'),
        tf.keras.layers.Activation('relu'),

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(200, use_bias=False),
        tf.keras.layers.Activation('relu'),
        tf.keras.layers.Dropout(0.4), 

        tf.keras.layers.Dense(10, activation='softmax')
      ])
  model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, ), loss = 'categorical_crossentropy', metrics = ['accuracy'])
  return model


In [12]:
with strategy.scope():
  model = create_model()
  steps_per_epoch = 60000//BATCH_SIZE  # 60,000 items in this dataset
  start_time = time.time()  
  model.fit(training_dataset,
            batch_size = BATCH_SIZE,
            epochs=20,
            steps_per_epoch=steps_per_epoch)
  end_time = time.time()
print(f"Time taken to train DL model using TPU: {end_time-start_time}s")

Epoch 1/20
58/58 [==============================] - 5s 21ms/step - loss: 0.8114 - accuracy: 0.7350
Epoch 2/20
58/58 [==============================] - 1s 23ms/step - loss: 0.0995 - accuracy: 0.9708
Epoch 3/20
58/58 [==============================] - 1s 23ms/step - loss: 0.0667 - accuracy: 0.9803
Epoch 4/20
58/58 [==============================] - 1s 23ms/step - loss: 0.0480 - accuracy: 0.9869
Epoch 5/20
58/58 [==============================] - 1s 23ms/step - loss: 0.0348 - accuracy: 0.9885
Epoch 6/20
58/58 [==============================] - 1s 23ms/step - loss: 0.0303 - accuracy: 0.9905
Epoch 7/20
58/58 [==============================] - 1s 22ms/step - loss: 0.0257 - accuracy: 0.9919
Epoch 8/20
58/58 [==============================] - 1s 22ms/step - loss: 0.0217 - accuracy: 0.9931
Epoch 9/20
58/58 [==============================] - 1s 22ms/step - loss: 0.0192 - accuracy: 0.9941
Epoch 10/20
58/58 [==============================] - 1s 22ms/step - loss: 0.0171 - accuracy: 0.9944
Epoch 11/

In [13]:
final_stats = model.evaluate(validation_dataset, steps=1)
print("Validation accuracy: ", final_stats[1])

1/1 [==============================] - 2s 2s/step - loss: 0.0314 - accuracy: 0.9920
Validation accuracy:  0.9919999837875366
